# GPT Codding

Zero to Hero

In [2]:
# read Nutuk in to inspect it
with open('nutuk.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("Lenght of dataset in characters: ", len(text))

Lenght of dataset in characters:  1577732


In [4]:
print(text[9000:10000])

sında
Diyarbekir (Vesika: 8, 9), Bitlis, Elaziz vilayetlerin­
de, İstanbul'dan idare olunan Kürt Teafi Cemiyeti!
vardı. Bu cemiyetin maksadı, yabancı himayesi altında bir Kürt hükümeti vü­
cuda getirmekti.
Konya ve havalisinde, İstanbul'dan idare olunan Teaiii İslam Cemiyeti teş­
kiline çalışılıyordu. Memleketin hemen her tarafında İtilaf ve Hürriyet, Sulh
ve Selamet cemiyetleri de vardı.

Memleket dahilinde
ve İstanbul'da milli
varlığa düşman
teşekküller

İngiliz Muhipleri
Cemiyeti

İstanbul'da, muhtelif maksatlarla gizli ve açık olmak
üzere de birtakım fırka veya cemiyet unvanı altında te­
şekküller vardı.
İstanbul'da mühim sayılacak teşebbüslerden biri İngiliz Muhipleri Cemiyeti
idi. Bu isimden, İngilizlere muhip2 olanların teşkil ettiği bir cemiyet anlaşılma­
sın! Bence, bu cemiyeti teşkil edenler, kendi şahıslannı ve şahsi menfaatlannı
sevenler ve şahıslanyla menfaatlannın dokunulmazlığı çaresini Loyd Core3 hü­
kümeti marifetiyle İngiliz himayesini teminde arayanlardır. Bu bedbaht

In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"%'()*,-./0123456789:;<>?ABCDEFGHIJKLMNOPQRSTUVWYZ[\]_abcdefghijklmnopqrstuvwxyz{§«­·ÇÖÜçôöüğİıŞş•�
103


In [6]:
# tokenization

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode('demir cucu'))
print(decode([61, 62, 70, 66, 75, 2, 60, 78, 60, 78]))

[61, 62, 70, 66, 75, 2, 60, 78, 60, 78]
demir cucu


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch
import torch.nn as nn
from torch.nn import functional as F

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[9000:10000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1577732]) torch.int64
tensor([ 76,  98,  71,  61,  58,   0,  32,  66,  82,  58,  75,  59,  62,  68,
         66,  75,   2,   7,  50,  62,  76,  66,  68,  58,  24,   2,  22,  10,
          2,  23,   8,  10,   2,  30,  66,  77,  69,  66,  76,  10,   2,  33,
         69,  58,  83,  66,  83,   2,  79,  66,  69,  58,  82,  62,  77,  69,
         62,  75,  66,  71,  87,   0,  61,  62,  10,   2,  97,  76,  77,  58,
         71,  59,  78,  69,   6,  61,  58,  71,   2,  66,  61,  58,  75,  62,
          2,  72,  69,  78,  71,  58,  71,   2,  39,  95,  75,  77,   2,  48,
         62,  58,  63,  66,   2,  31,  62,  70,  66,  82,  62,  77,  66,   3,
          0,  79,  58,  75,  61,  98,  12,   2,  30,  78,   2,  60,  62,  70,
         66,  82,  62,  77,  66,  71,   2,  70,  58,  68,  76,  58,  61,  98,
         10,   2,  82,  58,  59,  58,  71,  60,  98,   2,  65,  66,  70,  58,
         82,  62,  76,  66,   2,  58,  69,  77,  98,  71,  61,  58,   2,  59,
         66,  75,   2,  39,  9

In [8]:
# let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
# block_size is the maximum context length the model will be able to learn
block_size = 8
train_data[:block_size+1]

tensor([42, 49, 48, 49, 39, 37,  0,  0,  7])

In [10]:
print(train_data[:block_size*2])
x = train_data[:block_size*2]
y = train_data[:block_size*2+1]
for t in range(block_size):
    context = x[t:block_size+t+1]
    target = y[block_size+t]
    print(f"when input is {context} the target: {target}")

tensor([42, 49, 48, 49, 39, 37,  0,  0,  7, 15, 23, 15, 23, 11, 15, 23])
when input is tensor([42, 49, 48, 49, 39, 37,  0,  0,  7]) the target: 7
when input is tensor([49, 48, 49, 39, 37,  0,  0,  7, 15]) the target: 15
when input is tensor([48, 49, 39, 37,  0,  0,  7, 15, 23]) the target: 23
when input is tensor([49, 39, 37,  0,  0,  7, 15, 23, 15]) the target: 15
when input is tensor([39, 37,  0,  0,  7, 15, 23, 15, 23]) the target: 23
when input is tensor([37,  0,  0,  7, 15, 23, 15, 23, 11]) the target: 11
when input is tensor([ 0,  0,  7, 15, 23, 15, 23, 11, 15]) the target: 15
when input is tensor([ 0,  7, 15, 23, 15, 23, 11, 15, 23]) the target: 23


In [11]:
torch.manual_seed(42)
batch_size = 4 # how many independent sequence will we process in parallel
black_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} the target: {target}')

inputs:
torch.Size([4, 8])
tensor([[ 12,   2,  30,  58,  75,  98, 100,   2],
        [ 58,  69,  98, 100,  98,  69,  70,  98],
        [ 58,  82,  66,  70,   2,  79,  62,   2],
        [ 70,  98, 100,  77,  98,  75,  12,   2]])
targets:
torch.Size([4, 8])
tensor([[  2,  30,  58,  75,  98, 100,   2,  68],
        [ 69,  98, 100,  98,  69,  70,  98, 100],
        [ 82,  66,  70,   2,  79,  62,   2,  71],
        [ 98, 100,  77,  98,  75,  12,   2,  97]])
----
when input is [[12, 2, 30, 58, 75, 98, 100, 2]] the target: 2
when input is [[12, 2, 30, 58, 75, 98, 100, 2], [58, 69, 98, 100, 98, 69, 70, 98]] the target: 30
when input is [[12, 2, 30, 58, 75, 98, 100, 2], [58, 69, 98, 100, 98, 69, 70, 98], [58, 82, 66, 70, 2, 79, 62, 2]] the target: 58
when input is [[12, 2, 30, 58, 75, 98, 100, 2], [58, 69, 98, 100, 98, 69, 70, 98], [58, 82, 66, 70, 2, 79, 62, 2], [70, 98, 100, 77, 98, 75, 12, 2]] the target: 75
when input is [[12, 2, 30, 58, 75, 98, 100, 2], [58, 69, 98, 100, 98, 69, 70, 98], [

# Token Embedding

## Token Embedding

Her bir tokenin temsil ettigi feature vector olacak.
Yazacagimiz kodun init fonksiyonu:

```
def __init__(self):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd) # final layer norm
    self.lm_head = nn.Linear(n_embd, vocab_size)

    # better init, not covered in the original GPT video, but important, will cover in followup video
    self.apply(self._init_weights)
```

Vocab_size imiz kadar row, n_embd kadar da column. Her bir karakter/token'in kendine ait ogrenilebilir bir degeri olmasi gerekiyor. Yukarida tokenleri sayi indekslerine cevirdik ancak mesela 'bosluk' karakteri ile '!' karakterleri ikisi de tek bir integer olarak ifade edilse, modelin 'bosluk' karakterinin degerini anlamasi cok zor. Ancak bunlari modelin ogrenebilecegi buyuk bir vektor ile ifade edersek, bu sefer karakterler kendi basina da bayagi bir anlam ifade edecektir.

# Weighted aggregation in self-attention

d e m i r ' ' c u c u
3 2 6 4 6  7  4 2 4 5

In [12]:
v_value = (4 + (3+2+6+4+6+7))/7
v_value

4.571428571428571

In [13]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
print(x.shape)
print(x[0])

torch.Size([4, 8, 2])
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])


In [14]:
# use softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow = wei @ x # bag of words
print(wei)
print(x[0])
print(xbow[0])

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.09

# Self-Attention

key, query, value

d e m i r ' ' c u c u

d -> key_d, query_d, value_d
e -> key_e, query_e, value_e
...

Query: Ne arıyorum?
Key: Neye sahibim?
Value: Değerim ne?

wei = query.key
wei*value

In [21]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # maskelememiz lazım çünkü geleceğe bakamayız + encoder yapmak için tek silmemiz gereken satır
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

print(out.shape)
print(out[0])

torch.Size([4, 8, 16])
tensor([[-0.1571,  0.8801,  0.1615, -0.7824, -0.1429,  0.7468,  0.1007, -0.5239,
         -0.8873,  0.1907,  0.1762, -0.5943, -0.4812, -0.4860,  0.2862,  0.5710],
        [ 0.6764, -0.5477, -0.2478,  0.3143, -0.1280, -0.2952, -0.4296, -0.1089,
         -0.0493,  0.7268,  0.7130, -0.1164,  0.3266,  0.3431, -0.0710,  1.2716],
        [ 0.4823, -0.1069, -0.4055,  0.1770,  0.1581, -0.1697,  0.0162,  0.0215,
         -0.2490, -0.3773,  0.2787,  0.1629, -0.2895, -0.0676, -0.1416,  1.2194],
        [ 0.1971,  0.2856, -0.1303, -0.2655,  0.0668,  0.1954,  0.0281, -0.2451,
         -0.4647,  0.0693,  0.1528, -0.2032, -0.2479, -0.1621,  0.1947,  0.7678],
        [ 0.2510,  0.7346,  0.5939,  0.2516,  0.2606,  0.7582,  0.5595,  0.3539,
         -0.5934, -1.0807, -0.3111, -0.2781, -0.9054,  0.1318, -0.1382,  0.6371],
        [ 0.3428,  0.4960,  0.4725,  0.3028,  0.1844,  0.5814,  0.3824,  0.2952,
         -0.4897, -0.7705, -0.1172, -0.2541, -0.6892,  0.1979, -0.1513,  0.7666],

In [22]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
# Bizim durumumuzda gecmisteki nodelar gelecekteki nodelar hakkinda bilgiye sahip degil. Ancak bu her zaman boyle olmak zorunda degil.
# Bazi mimarilerde nodelarin hepsinin birbiriyle aggregationina bakilabilir.
# Bizim yaptigimiza decoder block, hepsinin birbiriyle iletisim kurduguna da encoder block deniyor.
# Dolayisiyla bu yuzden decoder-only bir mimari ile devam edecegiz.

# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>

In [24]:
n_embd = 256 # Embedding dimension (hidden size). Each token is mapped to a vector of size 256.
dropout = 0.2 # Dropout rate for regularization.

class Head(nn.Module):
    """ one head of self-attention """
    """ Scaled Dot-Products Attention in the paper """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout) # Her forward pass'te bazı neuronları rastgele sıfırlarız. Regularization tekniği.

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head_size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.values(x)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    """ Multi-Head Attention in the paper"""

    def __init__(self, num_heads, head_size):
        super.__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [25]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    """ Position-wise Feed-Forward Networks in the paper """
    # Tokenlerin birbiri ile iletişiminden sonra 'düşünecek bir zaman'a ihtiyaçları var. Onun gerçekleştirildiği layer.

    def __init__(self, n_embd):
        super().__ini__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [26]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size) # -> communication
        self.ffwd = FeedForward(n_embd) # -> computation

        self.ln1 = nn.LayerNorm(n_embd) # LayerNorm ensures that the added vectors (original + transformed) are on compatible scales.   
        self.ln2 = nn.LayerNorm(n_embd) # By normalizing across features per token, gradients don't get stuck due to extreme values.
        # Bun normalde paperda multihead attention ve feedforward layerlardan sonra yapılıyor ama artık öncesinde yapmak daha yaygın. #

        def forward(self, x):
            # x = self.sa(self.ln1(x))
            # x = self.ffwd(self.ln2(x))
            x = x + self.sa(self.ln1(x)) # -> Residual connection
            x = x + self.ffwd(self.ln2(x)) # -> Residual connection
            return x

In [27]:
n_head = 6 # Number of attention heads in multi-head self-attention.
n_layer = 6 # Number of Transformer blocks stacked.

class GPTLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from  a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self.init_weights)

    def init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.block(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) #(B, T+1)
        return idx 